# 🚀 Mejora del modelo de predicción de salario en IT

Aplicamos varias estrategias para mejorar el modelo de predicción de salario en el sector tecnológico:
1. Filtrado de salarios extremos
2. Logaritmo del salario
3. Expansión de columnas multietiqueta
4. Agrupación de países
5. LightGBM con RandomizedSearchCV


In [ ]:

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.metrics import r2_score
from lightgbm import LGBMRegressor
import warnings
warnings.filterwarnings("ignore")

# Cargar datos
df = pd.read_csv("survey_results_public.csv", low_memory=False)
columnas = {
    "Employment": "Tipo_empleo",
    "RemoteWork": "Trabajo_remoto",
    "DevType": "Rol",
    "EdLevel": "Nivel_educativo",
    "YearsCodePro": "Anios_experiencia",
    "Country": "Pais",
    "OrgSize": "Tamano_empresa",
    "ConvertedCompYearly": "Salario_anual"
}
df = df[list(columnas.keys())].rename(columns=columnas)

# Filtrar outliers
df = df[df["Salario_anual"].between(10000, 300000)]

# Experiencia
df["Anios_experiencia"] = df["Anios_experiencia"].replace({
    "Less than 1 year": "0", "More than 50 years": "51"
})
df["Anios_experiencia"] = pd.to_numeric(df["Anios_experiencia"], errors="coerce")

# Agrupar países
top_paises = df["Pais"].value_counts().head(15).index
df["Pais"] = df["Pais"].apply(lambda x: x if x in top_paises else "Otro")

# Expandir roles
roles = df["Rol"].str.get_dummies(sep=";")
df = pd.concat([df.drop(columns=["Rol"]), roles], axis=1)
df = df.dropna()

# Log del salario
df["Salario_anual_log"] = np.log1p(df["Salario_anual"])

# Separar variables
X = df.drop(columns=["Salario_anual", "Salario_anual_log"])
y = df["Salario_anual_log"]

cat_cols = X.select_dtypes(include="object").columns.tolist()
num_cols = X.select_dtypes(include="number").columns.tolist()

cat_pipeline = Pipeline([
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("encoder", OneHotEncoder(handle_unknown="ignore"))
])

num_pipeline = Pipeline([
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler", StandardScaler())
])

preprocessor = ColumnTransformer([
    ("cat", cat_pipeline, cat_cols),
    ("num", num_pipeline, num_cols)
])

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

param_dist = {
    "regressor__n_estimators": [100, 200, 300],
    "regressor__max_depth": [5, 10, 15, -1],
    "regressor__learning_rate": [0.01, 0.05, 0.1, 0.2],
    "regressor__num_leaves": [20, 31, 50, 100],
    "regressor__min_child_samples": [10, 20, 30]
}

pipe = Pipeline([
    ("preprocessor", preprocessor),
    ("regressor", LGBMRegressor(random_state=42))
])

search = RandomizedSearchCV(
    pipe,
    param_distributions=param_dist,
    n_iter=20,
    scoring="r2",
    cv=3,
    verbose=2,
    n_jobs=-1,
    random_state=42
)

search.fit(X_train, y_train)
print("Mejores parámetros encontrados:", search.best_params_)
print("Mejor R² en validación cruzada:", search.best_score_)

# Evaluar en test
mejor_modelo = search.best_estimator_
y_pred = mejor_modelo.predict(X_test)
print("R² en test:", r2_score(y_test, y_pred))
